In [ ]:
import cv2
from deepface import DeepFace
from tqdm import tqdm  # Biblioteca para a barra de progresso

def process_and_annotate_video(input_video_path, output_video_path):
    # Carregar o vídeo de entrada
    cap = cv2.VideoCapture(input_video_path)
    
    # Obter informações do vídeo de entrada
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Definir o codec e criar o objeto de gravação de vídeo de saída
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Carregar classificadores de rostos frontais e de perfil
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')
    
    # Barra de progresso configurada com o total de frames
    with tqdm(total=total_frames, desc="Processando vídeo", unit="frame") as pbar:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Converter o frame para escala de cinza para a detecção
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Detecção de rostos frontais
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            
            # Detecção de rostos de perfil
            profiles = profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            
            # Combinar rostos frontais e de perfil para a análise emocional
            all_faces = list(faces) + list(profiles)
            
            for (x, y, w, h) in all_faces:
                # Recortar a região do rosto
                face_region = frame[y:y+h, x:x+w]
                
                try:
                    # Análise de emoções para cada rosto detectado
                    result = DeepFace.analyze(face_region, actions=['emotion'], enforce_detection=False)
                    
                    # Verifique se `result` é uma lista e obtenha o primeiro item, se necessário
                    if isinstance(result, list):
                        result = result[0]  # Obter o primeiro item, se for uma lista
                    
                    # Acessar a emoção dominante
                    emotion = result.get('dominant_emotion', 'Desconhecido')
                    
                    # Desenhar a caixa delimitadora no rosto detectado
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    
                    # Adicionar o texto da emoção dominante
                    cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
                
                except Exception as e:
                    print("Erro ao analisar a emoção:", e)
            
            # Gravar o frame anotado no vídeo de saída
            out.write(frame)
            
            # Atualizar a barra de progresso
            pbar.update(1)
    
    # Liberação dos objetos de captura e gravação
    cap.release()
    out.release()
    print("Processamento concluído. O vídeo anotado foi salvo em:", output_video_path)

# Caminho do vídeo de entrada e de saída
input_video_path = "Video_tc.mp4"
output_video_path = "SAIDA.avi"

# Executar a função para processar o vídeo
process_and_annotate_video(input_video_path, output_video_path)
